# Trabalho Grafos 01

# instalação de  bibliotecas com pip

In [ ]:
%pip install spotipy
%pip install networkx
%pip install matplotlib
%pip install scipy

# biblioteca Spotyfy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import networkx as nx
import matplotlib.pyplot as plt

# Configurar as credenciais da API do Spotify
CLIENT_ID = 'f216308e304c4ba3be3fb4ce0f23ceb9'
CLIENT_SECRET = '33c84036de384fcc914b82a173cea2dc'

# Configurar a autenticação OAuth do Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri="http://localhost:8888/callback"))
